In [ ]:
!pip install transformers==4.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 68.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=de68a2828059e744f46379e607a78d33dc68de3bb47db0abf8be16fbfe5e94b7
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


In [ ]:
import re
import json
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prose

In [ ]:
import os
proza = list()
for root, dirs, files in os.walk('/content/drive/MyDrive/Diploma/prose'):
    for file in files:
        if file.endswith('.txt'):
          with open('/content/drive/MyDrive/Diploma/prose/'+file, 'r') as file_:
            data = file_.read().replace('\n', '').replace('\xa0', '') 
            proza.append(data)

In [ ]:
prz = pd.DataFrame(proza)

train, test = train_test_split(prz[0],test_size=0.15) 

def build_text_files(dataset, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for i in range(len(dataset)):
        poem = dataset.tolist()[i]
        data += poem + "  "
    f.write(data)


build_text_files(train,'/content/drive/MyDrive/Diploma/train_proza.txt')
build_text_files(test,'/content/drive/MyDrive/Diploma/test_proza.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

compete_dataset = prz[0]
build_text_files(compete_dataset,'/content/drive/MyDrive/Diploma/complete_proza.txt')

train_path = '/content/drive/MyDrive/Diploma/train_proza.txt'
test_path = '/content/drive/MyDrive/Diploma/test_proza.txt'

from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

complete_path = '/content/drive/MyDrive/Diploma/complete_proza.txt'

from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset_c(complete_path):
    complete_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=complete_path,
          block_size=128)
    return complete_dataset

complete_dataset = load_dataset_c(complete_path)

Train dataset length: 228
Test dataset length: 41


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Diploma/NN/PushkinGPT_prosetrained4", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps = 200, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=200,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:921: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Diploma/NN/PushkinGPT_prosetrained_n1", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=1, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps = 200, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=200,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=complete_dataset,
)

In [ ]:
len(list(model.parameters()))

148

In [ ]:
c=0
for param in model.parameters():
    param.requires_grad = False
    c+=1
    if c >146:
        break

In [ ]:
trainer.train()

Step,Training Loss
500,3.982400
1000,3.982200
1500,3.979300
2000,3.978500
2500,3.965700
3000,3.978300
3500,3.971200
4000,3.971000
4500,3.972400
5000,3.973100


TrainOutput(global_step=8897, training_loss=3.9727875678555624, metrics={'train_runtime': 1046.5781, 'train_samples_per_second': 8.501, 'total_flos': 6845556864319488, 'epoch': 1.0})

In [ ]:
trainer.save_model()

#Poetry

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Diploma/poems.xlsx')
df

,Unnamed: 0,author,title,poem,year,tags
0,0,Александр Пушкин,Письмо Татьяны к Онегину (отрывок из романа «Е...,['Я к вам пишу — чего же боле?\nЧто я могу еще...,1833.0,"['О любви', 'Золотой век', 'Стихи Александра П..."
1,1,Константин Симонов,"Жди меня, и я вернусь…","['Жди меня, и я вернусь.\nТолько очень жди,\nЖ...",1941.0,"['О войне', 'О родине', 'О любви', 'Для детей'..."
2,2,Сергей Есенин,Письмо к женщине,"['Вы помните,\nВы всё, конечно, помните,\nКак ...",1924.0,"['Для детей', 'О женщине', 'О любви', 'О жизни..."
3,3,Владимир Высоцкий,У неё всё своё,"['У неё всё своё — и бельё, и жильё,\nНу, а я ...",NaN,"['О женщине', 'О любви', 'Советские', 'Стихи В..."
4,4,Владимир Высоцкий,"Вот она, вот она…","['Вот она, вот она -\n Наших душ глубина,\n В ...",1978.0,"['О любви', 'Советские', 'Стихи Владимира Высо..."
...,...,...,...,...,...,...
4821,4821,Александр Грибоедов,К Дориде,"['Дорида, Дорида! любовью все дышит,\nВсе пьет...",1815.0,"['О любви', 'Длинные', 'Стихи Александра Грибо..."
4822,4822,Иннокентий Анненский,К Гретхен,['Во время представления «Le petit Faust» {*}\...,1869.0,"['О любви', 'Длинные', 'Серебряный век', 'Стих..."
4823,4823,Александр Грибоедов,В альбом (Не мило мне на новоселье),"['Не мило мне на новоселье,\nЗдесь все увяло, ...",1817.0,"['О жизни', 'О любви', 'Длинные', 'Стихи Алекс..."
4824,4824,Иннокентий Анненский,Е.А. Хвостовой экспромт,"['Добры к поэтам молодым,\nВы каждым опытом мо...",1858.0,"['О любви', 'Длинные', 'Серебряный век', 'Стих..."


In [ ]:
df = df.drop([8,72,119,2828]).reset_index()
df

,index,Unnamed: 0,author,title,poem,year,tags
0,0,0,Александр Пушкин,Письмо Татьяны к Онегину (отрывок из романа «Е...,['Я к вам пишу — чего же боле?\nЧто я могу еще...,1833.0,"['О любви', 'Золотой век', 'Стихи Александра П..."
1,1,1,Константин Симонов,"Жди меня, и я вернусь…","['Жди меня, и я вернусь.\nТолько очень жди,\nЖ...",1941.0,"['О войне', 'О родине', 'О любви', 'Для детей'..."
2,2,2,Сергей Есенин,Письмо к женщине,"['Вы помните,\nВы всё, конечно, помните,\nКак ...",1924.0,"['Для детей', 'О женщине', 'О любви', 'О жизни..."
3,3,3,Владимир Высоцкий,У неё всё своё,"['У неё всё своё — и бельё, и жильё,\nНу, а я ...",NaN,"['О женщине', 'О любви', 'Советские', 'Стихи В..."
4,4,4,Владимир Высоцкий,"Вот она, вот она…","['Вот она, вот она -\n Наших душ глубина,\n В ...",1978.0,"['О любви', 'Советские', 'Стихи Владимира Высо..."
...,...,...,...,...,...,...,...
4817,4821,4821,Александр Грибоедов,К Дориде,"['Дорида, Дорида! любовью все дышит,\nВсе пьет...",1815.0,"['О любви', 'Длинные', 'Стихи Александра Грибо..."
4818,4822,4822,Иннокентий Анненский,К Гретхен,['Во время представления «Le petit Faust» {*}\...,1869.0,"['О любви', 'Длинные', 'Серебряный век', 'Стих..."
4819,4823,4823,Александр Грибоедов,В альбом (Не мило мне на новоселье),"['Не мило мне на новоселье,\nЗдесь все увяло, ...",1817.0,"['О жизни', 'О любви', 'Длинные', 'Стихи Алекс..."
4820,4824,4824,Иннокентий Анненский,Е.А. Хвостовой экспромт,"['Добры к поэтам молодым,\nВы каждым опытом мо...",1858.0,"['О любви', 'Длинные', 'Серебряный век', 'Стих..."


In [ ]:
df.poem = df[['poem']].applymap(lambda x: eval(x))

In [ ]:
df['pretty_poem'] = df[['poem']].applymap(lambda alpha: " ".join(alpha).replace("\n"," ").replace("\xa0","").replace("  "," "))
df['pretty_poem']

0       Я к вам пишу — чего же боле? Что я могу еще ск...
1       Жди меня, и я вернусь. Только очень жди, Жди, ...
2       Вы помните, Вы всё, конечно, помните, Как я ст...
3       У неё всё своё — и бельё, и жильё, Ну, а я анг...
4       Вот она, вот она - Наших душ глубина, В ней дв...
                              ...                        
4817    Дорида, Дорида! любовью все дышит, Все пьет на...
4818    Во время представления «Le petit Faust» {*} {*...
4819    Не мило мне на новоселье, Здесь все увяло, там...
4820    Добры к поэтам молодым, Вы каждым опытом моим ...
4821    (В альбом)Пять звезд увенчали чело вдохновенно...
Name: pretty_poem, Length: 4822, dtype: object

In [ ]:
df['pretty_poem'].tolist()[0]

'Я к вам пишу — чего же боле? Что я могу еще сказать? Теперь, я знаю, в вашей воле Меня презреньем наказать. Но вы, к моей несчастной доле Хоть каплю жалости храня, Вы не оставите меня. Сначала я молчать хотела; Поверьте: моего стыда Вы не узнали б никогда, Когда б надежду я имела Хоть редко, хоть в неделю раз В деревне нашей видеть вас, Чтоб только слышать ваши речи, Вам слово молвить, и потом Все думать, думать об одном И день и ночь до новой встречи. Но, говорят, вы нелюдим; В глуши, в деревне всё вам скучно, А мы… ничем мы не блестим, Хоть вам и рады простодушно. Зачем вы посетили нас? В глуши забытого селенья Я никогда не знала б вас, Не знала б горького мученья. Души неопытной волненья Смирив со временем (как знать?), По сердцу я нашла бы друга, Была бы верная супруга И добродетельная мать. Другой!.. Нет, никому на свете Не отдала бы сердца я! То в вышнем суждено совете… То воля неба: я твоя; Вся жизнь моя была залогом Свиданья верного с тобой; Я знаю, ты мне послан богом, До гро

In [ ]:
df['pretty_poem'].tolist()[50]

'До свиданья, друг мой, до свиданья. Милый мой, ты у меня в груди. Предназначенное расставанье Обещает встречу впереди. До свиданья, друг мой, без руки, без слова, Не грусти и не печаль бровей, — В этой жизни умирать не ново, Но и жить, конечно, не новей.'

In [ ]:
complete_path = '/content/drive/MyDrive/Diploma/сomplete_dataset2.txt'

from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset_c(complete_path):
    complete_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=complete_path,
          block_size=128)
    return complete_dataset

complete_dataset = load_dataset_c(complete_path)

the next step is to download the tokenizer, which we use. We use the tokenizer from the `german-gpt2` model on [huggingface](https://huggingface.co/anonymous-german-nlp/german-gpt2).

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = '/content/drive/MyDrive/Diploma/train_dataset2.txt'
test_path = '/content/drive/MyDrive/Diploma/test_dataset2.txt'

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


data = df['pretty_poem']

def build_text_files(dataset, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for i in range(len(dataset)):
        poem = dataset.tolist()[i]
        data += poem + "  "
    f.write(data)

train, test = train_test_split(data,test_size=0.15) 


build_text_files(train,'/content/drive/MyDrive/Diploma/train_dataset2.txt')
build_text_files(test,'/content/drive/MyDrive/Diploma/test_dataset2.txt')
build_text_files(data,'/content/drive/MyDrive/Diploma/сomplete_dataset2.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


Train dataset length: 4098
Test dataset length: 724


In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/Diploma/NN/PushkinGPT_prosetrained_n1")


training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Diploma/NN/PushkinGPT_uptainedonprose8", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 200, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=200,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=complete_dataset#,
#    eval_dataset=test_dataset,
)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:921: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
c=0
for param in model.parameters():
    param.requires_grad = False
    c+=1
    if c >80:
        break

In [ ]:
len(list(model.parameters()))

148

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss
500,4.533000


TrainOutput(global_step=765, training_loss=4.493848294526144, metrics={'train_runtime': 500.3552, 'train_samples_per_second': 1.529, 'total_flos': 2352126259888128, 'epoch': 3.0})

After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [ ]:
trainer.save_model()